In [38]:
!pip install -r requirement.txt

In [71]:
import numpy as np
import geatpy as ea
import pandas as pd
import random
from datetime import datetime
print(ea.__version__)

2.7.0


### READ DATA
TODO: read data from GP

In [2]:
rawData = pd.read_excel(io='./FCI ANZ (2022-07-08) (NBV as at 30 Jun 2022)_v2.xlsx', sheet_name='Raw (portfolio)', engine='openpyxl')

In [3]:
# randomly choose 1000 samples
data = rawData.iloc[:30000, :61].sample(1000)
print(data.shape)
data.sample(3)

(1000, 61)


,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,...,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units
27782,FZOPS,DF3RM,FDCU0194586,3682.0,DFIC-FL-1801,1/3/2018,1103142.0,DFC3,D4H,FDCU0194586,...,1.7,2018 NP,Reasonable,8.909164,11.490836,from available portfolio,LF-HAPAG-09D4H,NaN,2516.0,1.0
14464,FZOPS,DF3RM,FFAU4041720,6000.0,CSIH-FL-2115,4/8/2021,1707069.0,DFC3,D4H,FFAU4041720,...,1.7,2021 NP,Reasonable,3.367164,10.637808,from ANZ portfolio,LT-MSC-48D4H,NaN,6997.0,1.0
208,FZOPS,DF3RM,FSCU7165697,3832.0,CIMC-FL-1801,12/18/2017,1070020.0,DFC3,D4H,FSCU7165697,...,1.7,2018 NP,Reasonable,8.977164,11.422836,from ANZ portfolio,LF-HAPAG-09D4H,NaN,2516.0,1.0


### DATA PREPARATION

In [62]:
# TODO: add more columns later
input_data = \
    data.loc[:, ['Cost', 'Mfr Dt Fz', 'Product', 'Contract Cust Id', 'Contract Dt', 'Contract Expiration Dt', 'Nbv', 'Billing Status Fz']]
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
18669,6000.0,4/12/2021,D4H,MSC,2021-04-01,2029-06-30,5612.350,ON
13072,6007.0,4/28/2021,D4H,MSC,2021-04-01,2029-06-30,5591.070,ON
1481,3832.0,12/18/2017,D4H,HAPAG,2017-11-01,2030-01-31,2981.838,ON


In [63]:
# select standard equipment
standardEquipment = ['D4H', 'D20', 'D40']
def SelectStandardEquip(Product):
    if Product in standardEquipment:
        Product = 1
    else:
        Product = 0
    return Product

input_data['Product'] = input_data.apply(lambda x: SelectStandardEquip(x['Product']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
9667,4420.0,3/24/2021,1,ONE,2020-11-16,2029-05-15,4136.34,ON
27927,3682.0,1/3/2018,1,HAPAG,2017-11-01,2030-01-31,2869.54,ON
14897,6000.0,4/8/2021,1,MSC,2021-04-01,2029-06-30,5612.35,ON


Find Top 1 and Top 3 Lessees

In [64]:
# select the Top1 lessee
input_data.groupby('Contract Cust Id').count().sort_values('Cost', ascending=False)

,Cost,Mfr Dt Fz,Product,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
Contract Cust Id,,,,,,,
MSC,434,434,434,434,434,434,434
ONE,366,366,366,366,366,366,366
HAPAG,186,186,186,186,186,186,186
TCLC,6,6,6,6,6,6,6
CMA,4,4,4,4,4,4,4
COSMR,3,3,3,3,3,3,3
PANOCEAN,1,1,1,1,1,1,1


In [65]:
# top1 and top3 lessees 
Top1Lessee = 'MSC'
Top3Lessees = ['MSC', 'ONE', 'HAPAG']

#### Select top 1 and top 3 lessees
Assign new column `Top1Lessee` and `Top3Lessees`

In [66]:
def SelectTop1(custID):
    if custID == Top1Lessee:
        custID = 1
    else:
        custID = 0
    return custID

def SelectTop3(custID):
    if custID in Top3Lessees:
        custID = 1
    else:
        custID = 0
    return custID

input_data['Top1Lessee'] = input_data.apply(lambda x: SelectTop1(x['Contract Cust Id']), axis=1)
input_data['Top3Lessees'] = input_data.apply(lambda x: SelectTop3(x['Contract Cust Id']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees
7729,3702.0,6/11/2018,1,MSC,2018-03-01,2032-07-31,2983.214,ON,1,1
28503,3682.0,1/3/2018,1,HAPAG,2017-11-01,2030-01-31,2869.540,ON,0,1
11372,4420.0,3/24/2021,1,ONE,2020-11-16,2029-05-15,4155.250,ON,0,1


#### Calcualte manufacture age
Assign new column `MfrAge`

In [98]:
# TODO: consider timezone, consider time_now in the future
time_now = datetime.now()
# calculate manufacture age
date_format = '%m/%d/%Y'
# return in years, round to 2 decimal
def CalculateManufacutreAge(date):
    diff = time_now.date() - datetime.strptime(date, date_format).date()
    return round(diff.days / 365, 2)

input_data['MfrAge'] = input_data.apply(lambda x: CalculateManufacutreAge(x['Mfr Dt Fz']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge
452,3832.0,12/18/2017,1,HAPAG,2017-11-01,2030-01-31,2981.836,ON,0,1,4.76
10103,4420.0,3/24/2021,1,ONE,2020-11-16,2029-05-15,4136.340,ON,0,1,1.50
9183,5150.0,4/20/2021,1,MSC,2021-04-01,2029-06-30,4773.960,ON,1,1,1.42


#### Calculate unexpired year 
Assign new column `ExpYear`

In [112]:
input_data['Contract Expiration Dt'][1021]

Timestamp('2030-01-31 00:00:00')

In [117]:
# TODO: consider timezone, consider time_now in the future
# TODO: 'Contract Expiration Dt' is in Timestamp, the data type may change
time_now = datetime.now()
# calculate manufacture age
date_format = '%Y-%m-%d'
# return in years, round to 2 decimal
def CalculateExpYear(date):
    diff = pd.to_datetime(date,format="%Y%m%d").date() - time_now.date()
    return round(diff.days / 365, 2)

input_data['ExpYear'] = input_data.apply(lambda x: CalculateExpYear(x['Contract Expiration Dt']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
5334,3682.0,2/28/2018,1,MSC,2018-03-01,2032-07-31,2933.000,ON,1,1,4.57,9.86
13907,6007.0,4/28/2021,1,MSC,2021-04-01,2029-06-30,5618.800,ON,1,1,1.40,6.78
4097,3682.0,2/28/2018,1,MSC,2018-03-01,2032-07-31,2917.312,ON,1,1,4.57,9.86
